In [1]:
# Import ArcGIS package
import arcpy

# pandas to view LAS statistics
import pandas as pd

In [2]:
# Subprocess allows us to issue commands on the command line
import subprocess

In [3]:
# Create a variable of the output and then decode it.
completed = subprocess.run(f'dir',  shell=True, stdout=subprocess.PIPE)
print(completed.stdout.decode('UTF-8'))

 Volume in drive C is WINDOWS
 Volume Serial Number is 7A8F-990D

 Directory of C:\DylanGIS\rrg

12/09/2019  12:45 PM    <DIR>          .
12/09/2019  12:45 PM    <DIR>          ..
09/04/2019  09:55 AM                66 .gitattributes
09/04/2019  09:55 AM             1,694 .gitignore
12/07/2019  05:20 PM    <DIR>          .ipynb_checkpoints
10/24/2019  05:34 PM    <DIR>          info
12/09/2019  12:45 PM            44,447 Lab Seven.ipynb
12/09/2019  11:42 AM    <DIR>          LabSeven
09/04/2019  09:55 AM            35,129 LICENSE
12/03/2019  10:39 AM               216 readme.md
10/24/2019  05:34 PM           748,546 RedRiverGorge.aprx
09/10/2019  01:01 PM                 0 rrg
10/24/2019  05:34 PM         2,626,615 rrgMap.pdf
               8 File(s)      3,456,713 bytes
               5 Dir(s)  874,652,233,728 bytes free



In [4]:
# Directory for project 
out_directory = f'C:\\DylanGIS\\rrg\\'

# Project name - creates a folder in project directory
project = 'LabSeven'

# Name of point layer and buffer distance from point
point_name = 'LabSeven_pt'
buffer_distance = 1000

# Output geodatabase name
out_geodb = 'LabSeven.gdb'

# Locations of index grids
las_grid = f'C:\\DylanGIS\\data\\data_pointcloud_grid\\lidar_grid.gdb\\KY_5k_PointClound_grid'
naip_grid = f'C:\\DylanGIS\\data\\data_pointcloud_grid\\Kentucky_10K_NAIP.shp'

# NAIP files have prefix
naip_prefix = 'ky_2ft_naip_2016_'

# Point the script to the directory with the laszip64.exe
las_tools = [f'C:\\DylanGIS\\data\\data_pointcloud_grid\\', 'laszip64.exe']

# Downloads and lidar folders
downloads = f'{out_directory}{project}\\downloads\\'
lidar = f'{out_directory}{project}\\lidar\\'
lidar_color = f'{out_directory}{project}\\lidar_color\\'

# LAS dataset name, temp list, and class codes
las_dataset = f'{lidar}{project}.lasd'
las_ground = [2]

In [5]:
# Create folders and copy laszip64.exe
subprocess.run(f'mkdir {out_directory}{project}', shell=True)
subprocess.run(f'mkdir {out_directory}{project}\\downloads', shell=True)
subprocess.run(f'mkdir {out_directory}{project}\\lidar', shell=True)
subprocess.run(f'mkdir {out_directory}{project}\\lidar_color', shell=True)

CompletedProcess(args='mkdir C:\\DylanGIS\\rrg\\LabSeven\\lidar_color', returncode=1)

In [6]:
# Copy laszip64.exe tool
completed = subprocess.run(f'copy {las_tools[0]}{las_tools[1]} {downloads}', shell=True, stdout=subprocess.PIPE)
print(completed.stdout.decode('UTF-8'))

        1 file(s) copied.



In [7]:
arcpy.env.overwriteOutput = True

# Create project geodatabase

# Use ArcPY function to check if asset exists
if arcpy.Exists(f'{out_directory}{project}\\{out_geodb}'):
    # If it does, print out the location.
    print(f"{out_directory}{project}\\{out_geodb} geodatabase ready for action!")
else:
    # Else if it doesn't create it and print location
    arcpy.CreateFileGDB_management(f'{out_directory}{project}', out_geodb)
    print(f"Created: {out_geodb}")


# Create project default geodatabase
out_path = f'{out_directory}{project}\\{out_geodb}'
arcpy.env.workspace = out_path


# Create default spatial reference setting
spatial_reference = arcpy.Describe(las_grid).spatialReference

Created: LabSeven.gdb


In [9]:
# Create empty point layer
arcpy.CreateFeatureclass_management(out_path, point_name, "POINT", "#", "#", "#", spatial_reference)

<Result 'C:\\DylanGIS\\rrg\\LabSeven\\LabSeven.gdb\\LabSeven_pt'>

In [10]:
# Check for layer
featureclasses = arcpy.ListFeatureClasses()

for fc in featureclasses:
    numberOfRecords = arcpy.GetCount_management(fc)
    if numberOfRecords[0] == '1':
        print(f"{fc} is ready to go with {numberOfRecords} point!")
    else:
        print(f"Check {fc}. We need one point. We have {numberOfRecords}.")

Check LabSeven_pt. We need one point. We have 0.


In [10]:
# Buffer point
arcpy.Buffer_analysis(point_name, f'{point_name}_{buffer_distance}ft', buffer_distance)


# Create a temp layer to find which LAS files to download
arcpy.Intersect_analysis ([f'{point_name}_{buffer_distance}ft', las_grid], "temp")

<Result 'C:\\DylanGIS\\rrg\\LabSeven\\workspace.gdb\\temp'>

In [11]:
# Find URLs and download them and use laszip64.exe to convert 
cursor = arcpy.da.SearchCursor("temp", ['ftppath', 'LASVersion', 'Year'])
i = 0
las_names = []
for row in cursor:
    url = row[0]
    name = url[-12:]
    las_names.append((f'{url[-12:-4]}.laz', url))
    i += 1
print(las_names)

[('N106E351.laz', 'ftp://ftp.kymartian.ky.gov/kyaped/LAZ/N106E351.laz'), ('N107E351.laz', 'ftp://ftp.kymartian.ky.gov/kyaped/LAZ/N107E351.laz'), ('N107E352.laz', 'ftp://ftp.kymartian.ky.gov/kyaped/LAZ/N107E352.laz'), ('N106E351.laz', 'ftp://ftp.kymartian.ky.gov/kyaped/LAZ/N106E351.laz'), ('N106E352.laz', 'ftp://ftp.kymartian.ky.gov/kyaped/LAZ/N106E352.laz'), ('N107E352.laz', 'ftp://ftp.kymartian.ky.gov/kyaped/LAZ/N107E352.laz'), ('N106E351.laz', 'ftp://ftp.kymartian.ky.gov/kyaped/LAZ/N106E351.laz'), ('N106E351.laz', 'ftp://ftp.kymartian.ky.gov/kyaped/LAZ/N106E351.laz'), ('N106E352.laz', 'ftp://ftp.kymartian.ky.gov/kyaped/LAZ/N106E352.laz'), ('N106E352.laz', 'ftp://ftp.kymartian.ky.gov/kyaped/LAZ/N106E352.laz'), ('N107E351.laz', 'ftp://ftp.kymartian.ky.gov/kyaped/LAZ/N107E351.laz'), ('N107E351.laz', 'ftp://ftp.kymartian.ky.gov/kyaped/LAZ/N107E351.laz'), ('N107E352.laz', 'ftp://ftp.kymartian.ky.gov/kyaped/LAZ/N107E352.laz'), ('N107E352.laz', 'ftp://ftp.kymartian.ky.gov/kyaped/LAZ/N107E35

In [12]:
# Use laszip64.exe to convert 
cursor = arcpy.da.SearchCursor("temp", ['ftppath', 'LASVersion', 'Year'])
las_names = []
i = 0
for row in cursor:
    url = row[0]
    print(url)
    name = url[-12:]
    print(lidar)
    print(las_names)
    las_names.append(f'{lidar}{url[-12:-4]}.las')
    print(las_names)
    print(f'{downloads}{las_tools[1]} -i {downloads}{name} -o {las_names[i]}')
    completed = subprocess.run(f'{downloads}{las_tools[1]} -i {downloads}{name} -o {las_names[i]}', shell=True, stdout=subprocess.PIPE)
    print(completed.stdout.decode('UTF-8'))
    i += 1

ftp://ftp.kymartian.ky.gov/kyaped/LAZ/N106E351.laz
C:\DylanGIS\rrg\LabSeven\lidar\
[]
['C:\\DylanGIS\\rrg\\LabSeven\\lidar\\N106E351.las']
C:\DylanGIS\rrg\LabSeven\downloads\laszip64.exe -i C:\DylanGIS\rrg\LabSeven\downloads\N106E351.laz -o C:\DylanGIS\rrg\LabSeven\lidar\N106E351.las

ftp://ftp.kymartian.ky.gov/kyaped/LAZ/N107E351.laz
C:\DylanGIS\rrg\LabSeven\lidar\
['C:\\DylanGIS\\rrg\\LabSeven\\lidar\\N106E351.las']
['C:\\DylanGIS\\rrg\\LabSeven\\lidar\\N106E351.las', 'C:\\DylanGIS\\rrg\\LabSeven\\lidar\\N107E351.las']
C:\DylanGIS\rrg\LabSeven\downloads\laszip64.exe -i C:\DylanGIS\rrg\LabSeven\downloads\N107E351.laz -o C:\DylanGIS\rrg\LabSeven\lidar\N107E351.las

ftp://ftp.kymartian.ky.gov/kyaped/LAZ/N107E352.laz
C:\DylanGIS\rrg\LabSeven\lidar\
['C:\\DylanGIS\\rrg\\LabSeven\\lidar\\N106E351.las', 'C:\\DylanGIS\\rrg\\LabSeven\\lidar\\N107E351.las']
['C:\\DylanGIS\\rrg\\LabSeven\\lidar\\N106E351.las', 'C:\\DylanGIS\\rrg\\LabSeven\\lidar\\N107E351.las', 'C:\\DylanGIS\\rrg\\LabSeven\\lid


ftp://ftp.kymartian.ky.gov/kyaped/LAZ/N107E351.laz
C:\DylanGIS\rrg\LabSeven\lidar\
['C:\\DylanGIS\\rrg\\LabSeven\\lidar\\N106E351.las', 'C:\\DylanGIS\\rrg\\LabSeven\\lidar\\N107E351.las', 'C:\\DylanGIS\\rrg\\LabSeven\\lidar\\N107E352.las', 'C:\\DylanGIS\\rrg\\LabSeven\\lidar\\N106E351.las', 'C:\\DylanGIS\\rrg\\LabSeven\\lidar\\N106E352.las', 'C:\\DylanGIS\\rrg\\LabSeven\\lidar\\N107E352.las', 'C:\\DylanGIS\\rrg\\LabSeven\\lidar\\N106E351.las', 'C:\\DylanGIS\\rrg\\LabSeven\\lidar\\N106E351.las', 'C:\\DylanGIS\\rrg\\LabSeven\\lidar\\N106E352.las', 'C:\\DylanGIS\\rrg\\LabSeven\\lidar\\N106E352.las', 'C:\\DylanGIS\\rrg\\LabSeven\\lidar\\N107E351.las']
['C:\\DylanGIS\\rrg\\LabSeven\\lidar\\N106E351.las', 'C:\\DylanGIS\\rrg\\LabSeven\\lidar\\N107E351.las', 'C:\\DylanGIS\\rrg\\LabSeven\\lidar\\N107E352.las', 'C:\\DylanGIS\\rrg\\LabSeven\\lidar\\N106E351.las', 'C:\\DylanGIS\\rrg\\LabSeven\\lidar\\N106E352.las', 'C:\\DylanGIS\\rrg\\LabSeven\\lidar\\N107E352.las', 'C:\\DylanGIS\\rrg\\LabSeven\\

In [16]:
arcpy.CreateLasDataset_management (las_names, las_dataset, "#", "#", spatial_reference, True, True)

<Result 'C:\\DylanGIS\\rrg\\LabSeven\\lidar\\LabSeven.lasd'>

In [24]:
#arcpy.LasDatasetStatistics_management (las_dataset, "#", f'{out_directory}{project}\\stats.csv', "#", "#", "#")
with open(f'{out_directory}{project}\\stats.csv', encoding='utf-8') as csv:
    reader = pd.read_csv(csv)
    # Create pandas data frame that
    pdData = pd.DataFrame(reader)

pdData[pdData["Category"] == "ClassCodes"]

,Item,Category,Pt_Cnt,Percent,Z_Min,Z_Max,Intensity_Min,Intensity_Max,Synthetic_Pt_Cnt,Range_Min,Range_Max
12,1_Unclassified,ClassCodes,55325281.0,79.70,681.72,1329.68,0.0,255.0,0.0,NaN,NaN
13,2_Ground,ClassCodes,14060647.0,20.25,681.42,1253.38,0.0,255.0,0.0,NaN,NaN
14,7_Low_Point(noise),ClassCodes,33993.0,0.05,615.28,2307.31,0.0,255.0,0.0,NaN,NaN


In [25]:
# default ground classes
las_ground = [2]

# Filter for ground points and create DEM and hillshade
arcpy.MakeLasDatasetLayer_management (las_dataset, f'{lidar}ground', las_ground)
arcpy.LasDatasetToRaster_conversion (f'{lidar}ground', f'{project}_dem_5ft', "#", "#", "#", "#", 5)
arcpy.HillShade_3d(f'{project}_dem_5ft', f'{project}_hillshade', 270, 55)

<Result 'C:\\DylanGIS\\rrg\\LabSeven\\workspace.gdb\\LabSeven_hillshade'>

In [26]:
# Create a temp layer to find which NAIP files to download
arcpy.RasterDomain_3d (f'{project}_hillshade', 'domain', 'POLYGON')
arcpy.Intersect_analysis (['domain', naip_grid], "temp")

<Result 'C:\\DylanGIS\\rrg\\LabSeven\\workspace.gdb\\temp'>

In [27]:
# Find URLs, download them and extract 
cursor = arcpy.da.SearchCursor("temp", ['ftppath16', 'TileName'])
naip_names = []
for row in cursor:
    url = row[0]
    name = row[1]
    naip_names.append((name, url))
print(naip_names)

[('N053E176', 'ftp://ftp.kymartian.ky.gov/FSA_NAIP_2016_2FT/ky_2ft_naip_2016_N053E176.zip'), ('N054E176', 'ftp://ftp.kymartian.ky.gov/FSA_NAIP_2016_2FT/ky_2ft_naip_2016_N054E176.zip')]


In [28]:
# Find URLs, download them and extract 
cursor = arcpy.da.SearchCursor("temp", ['ftppath16', 'TileName'])
naip_names = []
for row in cursor:
    name = row[1]
    naip_names.append(name)
    arcpy.CopyRaster_management (f'{workspace}\\{naip_prefix}{name}.jpg', name)

rasterLayers = arcpy.ListRasters()

for rl in rasterLayers: 
    print(rl)

focal_stats_40ft
cliffs_over_40ft
LabSeven_dem_5ft
LabSeven_hillshade
N053E176
N054E176


In [29]:
print(out_path)
print(naip_names)

C:\DylanGIS\rrg\LabSeven\workspace.gdb
['N053E176', 'N054E176']


In [41]:
# If multiple NAIPs, then mosaic to new raster and clip
if len(naip_names) > 1:
    arcpy.MosaicToNewRaster_management (naip_names, out_path, "temp", "#", "#", "#", 3, "#", "#")
    arcpy.Clip_management ('temp', '#', f'{project}_naip', 'domain')
else:
    arcpy.Clip_management (naip_names[0], '#', f'{project}_naip', 'domain')

ExecuteError: ERROR 999999: Something unexpected caused the tool to fail. Contact Esri Technical Support (http://esriurl.com/support) to Report a Bug, and refer to the error help for potential solutions or workarounds.
Failed to execute (MosaicToNewRaster).


In [23]:
rasterLayers = arcpy.ListRasters()

for rl in rasterLayers: 
    print(rl)

focal_stats_40ft
cliffs_over_40ft
LabSeven_dem_5ft
LabSeven_hillshade
N053E176
N054E176


In [ ]:
# Extract LAS points in buffer and colorize
arcpy.ExtractLas_3d (las_dataset, lidar, f'{point_name}_{buffer_distance}ft', "#", "#", "_extract", "#", "#", True, f'{lidar}extractedTemp.lasd')
arcpy.ColorizeLas_3d (f'{lidar}extractedTemp.lasd', f'{project}_naip', 'RED Band_1; GREEN Band_2; BLUE Band_3', lidar_color, "_color", "#",  "#",  "#",  "#", True, f'{lidar_color}{project}_rgb.lasd')


In [ ]:
# Create DSM of cliffs over 40 feet in 40-ft diameter neighborhood from bare-earth DEM
neighborhood = arcpy.sa.NbrCircle(3,'CELL')
outFocalStat = arcpy.sa.FocalStatistics(f'{project}_dem_5ft', neighborhood, "RANGE")
outFocalStat.save("focal_stats_40ft")
cliffs_over_40ft = arcpy.sa.Con(outFocalStat > 40, outFocalStat)
cliffs_over_40ft.save("cliffs_over_40ft")